In [ ]:
!pip install nltk requests

In [ ]:
import requests
import re
import nltk
from collections import Counter
from typing import Optional

# Загружаем стоп-слова
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords


class HHVacancyAnalyzer:

    BASE_URL = "https://api.hh.ru"

    CITIES = {
        "Москва": 1,
        "Санкт-Петербург": 2,
        "Екатеринбург": 3,
        "Пермь": 72,
        "Россия": 113
    }

    CURRENCY_RATES = {
        "RUR": 1, "RUB": 1, "USD": 90, "EUR": 98,
        "KZT": 0.20, "UAH": 2.4, "BYR": 28, "UZS": 0.007
    }

    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
        })

        # Стоп-слова из NLTK (русские + английские)
        self.stop_words = set(stopwords.words('russian')) | set(stopwords.words('english'))

        # Добавляем дополнительные стоп-слова, специфичные для вакансий
        self.stop_words.update({
            'опыт', 'работа', 'работы', 'года', 'лет', 'год', 'компания', 'компании',
            'условия', 'требования', 'обязанности', 'знание', 'навыки', 'умение',
            'experience', 'work', 'working', 'years', 'year', 'company', 'team',
            'nbsp', 'amp', 'quot', 'bull', 'mdash'  # HTML entities
        })

    def search_vacancies(self, text: str, area_id: Optional[int] = None,
                         per_page: int = 100, pages: int = 2) -> list:
        """Поиск вакансий по тексту"""
        vacancies = []

        for page in range(pages):
            params = {
                "text": text,
                "per_page": per_page,
                "page": page,
                "only_with_salary": "true"
            }
            if area_id:
                params["area"] = area_id

            try:
                response = self.session.get(f"{self.BASE_URL}/vacancies", params=params)
                response.raise_for_status()
                data = response.json()
                vacancies.extend(data.get("items", []))

                if page >= data.get("pages", 0) - 1:
                    break

            except requests.RequestException as e:
                print(f"Ошибка запроса: {e}")
                break

        return vacancies

    def get_vacancy_details(self, vacancy_id: str) -> dict:
        """Получить подробную информацию о вакансии"""
        try:
            response = self.session.get(f"{self.BASE_URL}/vacancies/{vacancy_id}")
            response.raise_for_status()
            return response.json()
        except requests.RequestException:
            return {}

    def convert_salary_to_rub(self, salary: dict) -> Optional[float]:
        """Конвертация зарплаты в рубли"""
        if not salary:
            return None

        currency = salary.get("currency", "RUR")
        rate = self.CURRENCY_RATES.get(currency, 1)

        salary_from = salary.get("from")
        salary_to = salary.get("to")

        if salary_from and salary_to:
            return ((salary_from + salary_to) / 2) * rate
        elif salary_from:
            return salary_from * rate
        elif salary_to:
            return salary_to * rate

        return None

    def calculate_average_salary(self, vacancies: list) -> Optional[float]:
        """Расчёт средней зарплаты"""
        salaries = []

        for v in vacancies:
            salary_rub = self.convert_salary_to_rub(v.get("salary"))
            if salary_rub:
                salaries.append(salary_rub)

        if salaries:
            return sum(salaries) / len(salaries)
        return None

    def compare_salaries_by_city(self, vacancy_text: str = "Python разработчик"):
        """Сравнение зарплат по городам"""
        print(f"СРАВНЕНИЕ ЗАРПЛАТ: {vacancy_text}")

        results = {}

        for city_name, city_id in self.CITIES.items():
            vacancies = self.search_vacancies(vacancy_text, city_id, per_page=50, pages=2)
            avg_salary = self.calculate_average_salary(vacancies)
            results[city_name] = {
                "count": len(vacancies),
                "avg_salary": avg_salary
            }

            if avg_salary:
                print(f"\n{city_name}:")
                print(f"   Найдено вакансий: {len(vacancies)}")
                print(f"   Средняя з/п: {avg_salary:,.0f} ₽")
            else:
                print(f"\n{city_name}: нет данных о зарплате")

        # Рейтинг
        print("РЕЙТИНГ ПО СРЕДНЕЙ ЗАРПЛАТЕ:")
        sorted_results = sorted(
            [(k, v) for k, v in results.items() if v["avg_salary"]],
            key=lambda x: x[1]["avg_salary"],
            reverse=True
        )

        for i, (city, data) in enumerate(sorted_results, 1):
            print(f"  {i}. {city}: {data['avg_salary']:,.0f} ₽ ({data['count']} вакансий)")

        return results

    def extract_ngrams(self, text: str, n: int = 2) -> list:
        """Извлечение n-грамм из текста (с использованием NLTK стоп-слов)"""
        # Очистка текста
        text = re.sub(r'<[^>]+>', ' ', text)  # Удаление HTML тегов
        text = re.sub(r'&\w+;', ' ', text)     # Удаление HTML entities
        text = re.sub(r'[^\w\s-]', ' ', text.lower())
        text = re.sub(r'\d+', ' ', text)       # Удаление чисел

        words = text.split()

        # Фильтрация через NLTK стоп-слова
        words = [w for w in words if w not in self.stop_words and len(w) > 2]

        # Генерация n-грамм
        ngrams = []
        for i in range(len(words) - n + 1):
            ngram = ' '.join(words[i:i+n])
            ngrams.append(ngram)

        return ngrams

    def build_skills_frequency(self, vacancy_text: str = "Python разработчик",
                               n: int = 2, top: int = 30):
        """Построение частотного словаря навыков"""
        print(f"АНАЛИЗ НАВЫКОВ И КОМПЕТЕНЦИЙ: {vacancy_text}")

        vacancies = self.search_vacancies(vacancy_text, per_page=50, pages=3)
        print(f"Загружено вакансий: {len(vacancies)}")
        print(f"Используется {len(self.stop_words)} стоп-слов (NLTK: ru + en)")

        all_ngrams = []

        for i, vacancy in enumerate(vacancies[:50]):
            details = self.get_vacancy_details(vacancy["id"])

            if details:
                description = details.get("description", "")
                key_skills = details.get("key_skills", [])
                skills_text = " ".join([s.get("name", "") for s in key_skills])

                full_text = f"{description} {skills_text}"
                ngrams = self.extract_ngrams(full_text, n)
                all_ngrams.extend(ngrams)

            if (i + 1) % 10 == 0:
                print(f"  Обработано: {i + 1} вакансий...")

        # Подсчёт частоты
        frequency = Counter(all_ngrams)

        # Фильтрация технических терминов
        tech_keywords = {
            'python', 'django', 'flask', 'fastapi', 'sql', 'postgresql', 'mysql',
            'mongodb', 'redis', 'docker', 'kubernetes', 'git', 'linux', 'aws',
            'api', 'rest', 'graphql', 'ci', 'cd', 'agile', 'scrum', 'jira',
            'machine', 'learning', 'data', 'science', 'tensorflow', 'pytorch',
            'pandas', 'numpy', 'asyncio', 'celery', 'rabbitmq', 'kafka',
            'elasticsearch', 'javascript', 'react', 'vue', 'angular', 'html',
            'css', 'frontend', 'backend', 'fullstack', 'devops', 'microservices',
            'unit', 'test', 'testing', 'development', 'разработка', 'разработчик'
        }

        filtered_frequency = {
            k: v for k, v in frequency.items()
            if any(tech in k for tech in tech_keywords) or v >= 5
        }

        print(f"Топ-{top} {n}-грамм (навыки и компетенции):")
        print("-" * 50)

        top_ngrams = Counter(filtered_frequency).most_common(top)

        for i, (ngram, count) in enumerate(top_ngrams, 1):
            bar = "█" * min(count // 2, 20)
            print(f"  {i:2}. {ngram:<35} {count:3} {bar}")

        return dict(top_ngrams)


def run_task2():
    """Запуск задания 2"""
    analyzer = HHVacancyAnalyzer()

    # Поиск по запросу пользователя
    search_query = input("Введите название вакансии для поиска (Enter = Python developer): ").strip()
    if not search_query:
        search_query = "Python developer"

    vacancies = analyzer.search_vacancies(search_query)
    print(f"\nНайдено вакансий: {len(vacancies)}")

    avg_salary = analyzer.calculate_average_salary(vacancies)
    if avg_salary:
        print(f"Средняя зарплата: {avg_salary:,.0f} ₽")

    # Сравнение зарплат по городам
    analyzer.compare_salaries_by_city("Python разработчик")

    # Анализ n-грамм
    analyzer.build_skills_frequency("Python разработчик", n=2, top=25)



In [ ]:
if __name__ == "__main__":
    run_task2()


Введите название вакансии для поиска (Enter = Python developer): Python developer

Найдено вакансий: 200
Средняя зарплата: 180,876 ₽
СРАВНЕНИЕ ЗАРПЛАТ: Python разработчик

Москва:
   Найдено вакансий: 100
   Средняя з/п: 214,435 ₽

Санкт-Петербург:
   Найдено вакансий: 100
   Средняя з/п: 186,325 ₽

Екатеринбург:
   Найдено вакансий: 26
   Средняя з/п: 172,404 ₽

Пермь:
   Найдено вакансий: 10
   Средняя з/п: 173,250 ₽

Россия:
   Найдено вакансий: 100
   Средняя з/п: 179,859 ₽
РЕЙТИНГ ПО СРЕДНЕЙ ЗАРПЛАТЕ:
  1. Москва: 214,435 ₽ (100 вакансий)
  2. Санкт-Петербург: 186,325 ₽ (100 вакансий)
  3. Россия: 179,859 ₽ (100 вакансий)
  4. Пермь: 173,250 ₽ (10 вакансий)
  5. Екатеринбург: 172,404 ₽ (26 вакансий)
АНАЛИЗ НАВЫКОВ И КОМПЕТЕНЦИЙ: Python разработчик
Загружено вакансий: 150
Используется 375 стоп-слов (NLTK: ru + en)
  Обработано: 10 вакансий...
  Обработано: 20 вакансий...
  Обработано: 30 вакансий...
  Обработано: 40 вакансий...
  Обработано: 50 вакансий...
Топ-25 2-грамм (навыки и 